<a href="https://colab.research.google.com/github/201524495/201524495/blob/main/gpt2_rotten_tomatoes_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine‑tuning GPT‑2 for Sentiment Analysis (rotten tomatoes)

**작성일**: 2025-07-19 by Youngwoo Kimh (Credit : DSAIL Lab, SNU)

**목표**:  
- BERT를 활용해 downstream task를 수행하는 실습
- Rotten tomato dataset을 통해 영화평의 긍정/부정을 파악

---
> 본 노트북은 HD현대 실습을 위해 교육용 자료로서 준비되었으며, PyTorch와 HuggingFace Transformers 라이브러리를 사용합니다.

## 0. Setup

In [1]:
!pip -q install transformers datasets evaluate accelerate fsspec -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1. Imports

In [2]:
from datasets import load_dataset
from transformers import (GPT2TokenizerFast, GPT2ForSequenceClassification,
                          DataCollatorWithPadding, Trainer, TrainingArguments)
from peft import LoraConfig, get_peft_model
import evaluate, torch, numpy as np

## 2. Load rotten tomatoes dataset

In [3]:
# load rotten tomatoes dataset for sentiment analysis
raw = load_dataset("rotten_tomatoes")
print(raw)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


### Inspect samples

In [4]:
for i in range(3):
    print('LABEL:', raw['train'][i]['label'])
    print(raw['train'][i]['text'][:400])
    print('-'*40)

LABEL: 1
the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
----------------------------------------
LABEL: 1
the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .
----------------------------------------
LABEL: 1
effective but too-tepid biopic
----------------------------------------


## 3. Prepare Tokenizer and Tokenize dataset

In [5]:
# load tokenizer and add padding token [PAD]
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})

# tokenize all the sentences in the dataset
def tok(b): return tokenizer(b["text"], truncation=True)
data = raw.map(tok, batched=True, remove_columns=["text"]).rename_column("label", "labels")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

### Load Model

In [6]:
# load model for classification
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

# resize token_embeddings length (added [PAD] token)
model.resize_token_embeddings(len(tokenizer))

# define Lora(Low-rank adaptation) config
lora_cfg = LoraConfig(r=8, lora_alpha=16, target_modules=["c_attn","c_proj"])

# get gpt-2 model with Lora adapted
model = get_peft_model(model, lora_cfg)
model.config.pad_token_id = tokenizer.pad_token_id

# load model on gpu
model.to("cuda")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


PeftModel(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50258, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magni

### Data collator

In [7]:
# define data collator (make dataset model-friendly)
collator = DataCollatorWithPadding(tokenizer)

## 4. Training

In [8]:
# define metrics to select the best model while training
metric = evaluate.load("accuracy")
def acc(p): logits, labels = p; return metric.compute(predictions=np.argmax(logits, -1), references=labels)

# define training arguments
args = TrainingArguments(
    "gpt2-rt-sentiment",
    per_device_train_batch_size=4, # batch size for each gpu
    gradient_accumulation_steps=4, # the number of batches to update model simultaneously
    num_train_epochs=3, # total epochs
    logging_steps=100,
    fp16=True, # cast model precision to fp16 (half the memory compared to original fp32 model)
    report_to="none",
)

# define trainer class
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=data["train"], # training dataset
    eval_dataset=data["validation"], # evaluation dataset
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8),
    compute_metrics=acc,
)
trainer.train()

/tmp/ipython-input-8-1150080540.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,1.020100
200,0.702400
300,0.678800
400,0.645700
500,0.600900
600,0.542400
700,0.492300
800,0.474200
900,0.505000
1000,0.459400


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=1602, training_loss=0.547229565633519, metrics={'train_runtime': 385.0059, 'train_samples_per_second': 66.467, 'train_steps_per_second': 4.161, 'total_flos': 533981440770048.0, 'train_loss': 0.547229565633519, 'epoch': 3.0})

## 5. Evaluation

In [9]:
trainer.evaluate()

{'eval_runtime': 3.2234,
 'eval_samples_per_second': 330.703,
 'eval_steps_per_second': 41.571,
 'epoch': 3.0}

## 6. Inference demo

In [10]:

def predict(text):
    # tokenize input
    inputs = tokenizer(text, return_tensors='pt').to('cuda')

    # inference
    with torch.no_grad():
        logits = model(**inputs).logits

    # probability of each label (Positive, Negative)
    label = logits.argmax(-1).item()
    return 'POSITIVE' if label==1 else 'NEGATIVE'

# example sentence
examples = [
    'An amazing movie with stellar performances.',
    'It was a waste of two hours.'
]

for ex in examples:
    print(ex, '→', predict(ex))


An amazing movie with stellar performances. → POSITIVE
It was a waste of two hours. → NEGATIVE


In [12]:
review = '' # fill your review sentence
print(review, '→', predict(review))

hello → POSITIVE
